In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os
source_directory = '/content/drive/MyDrive/output'
destination_directory = '/content/food_v5/main'

if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

shutil.copytree(source_directory, destination_directory, dirs_exist_ok=True)

print(f"Files from {source_directory} have been copied to {destination_directory}")

Files from /content/drive/MyDrive/output have been copied to /content/food_v5/main


In [ ]:
rar_file_path = '/content/main.rar'
output_directory = '/content/food_v5/main'
!unrar x -y {rar_file_path} {output_directory}


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/main.rar

Creating    /content/food_v5/main/temp                                OK
Creating    /content/food_v5/main/temp/checkpoint                     OK
Creating    /content/food_v5/main/temp/checkpoint/flat_efficientnetB1  OK
Creating    /content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalCategory  OK
Creating    /content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalCategory/weights  OK
Extracting  /content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalCategory/weights/checkpoint       0%  OK 
Creating    /content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer  OK
Creating    /content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer/weights  OK
Extracting  /content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer/weights/checkpoint       0%  OK 
Creating    /conten

In [ ]:
import shutil
import os
source_directory = '/content/drive/MyDrive/food_v5'
destination_directory = '/content/food_v5'

if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

shutil.copytree(source_directory, destination_directory, dirs_exist_ok=True)

print(f"Files from {source_directory} have been copied to {destination_directory}")

Files from /content/drive/MyDrive/food_v5 have been copied to /content/food_v5


In [ ]:
rar_file_path = '/content/food_v5/final-dataset/final-dataset.rar'
output_directory = '/content/food_v5/final-dataset'
!unrar x -y {rar_file_path} {output_directory}

Streaming output truncated to the last 5000 lines.
Extracting  /content/food_v5/final-dataset/images/scallops/f101_836284.jpg      99%  OK 
Extracting  /content/food_v5/final-dataset/images/pork_chop/f101_836291.jpg      99%  OK 
Extracting  /content/food_v5/final-dataset/images/hummus/f101_836325.jpg      99%  OK 
Extracting  /content/food_v5/final-dataset/images/hot_and_sour_soup/f101_836337.jpg      99%  OK 
Extracting  /content/food_v5/final-dataset/images/crab_cakes/f101_836352.jpg      99%  OK 
Extracting  /content/food_v5/final-dataset/images/club_sandwich/f101_836401.jpg      99%  OK 
Extracting  /content/food_v5/final-dataset/images/deviled_eggs/f101_836409.jpg      99%  OK 
Extracting  /content/food_v5/final-dataset/images/baby_back_ribs/f101_836437.jpg      99%  OK 
Extracting  /content/food_v5/final-dataset/images/onion_rings/f101_83644.jpg      99%  OK 
Extracting  /content/food_v5/final-datas

________________________________________________________________________________

# Building Model

In [ ]:
!pip install import_ipynb ipynb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from ipynb.fs.defs.data_pipeline import EXPORTED as data_pipeline
from tensorflow import keras
import json
import import_ipynb
from pathlib import Path
from types import SimpleNamespace
import pandas as pd

# Loading builders for datasets

In [ ]:
RECIPES5K_DIR = "/content/food_v5/final-dataset/tfrecord/recipes5k/1.0.0"
FOOD101_DIR = "/content/food_v5/final-dataset/tfrecord/food101/1.1.0"
NUTRITION5K_DIR = "/content/food_v5/final-dataset/tfrecord/nutrition5k/1.0.0"


recipes5k_food101_builder = tfds.builder_from_directories([RECIPES5K_DIR, FOOD101_DIR])
nutrition5k_builder = tfds.builder_from_directory(NUTRITION5K_DIR)

In [ ]:
recipes5k_food101_builder.info.splits["train"].num_examples

105826

In [ ]:
nutrition5k_builder.info.splits["train"].num_examples

271829

In [ ]:
def encode_category_ingredients(category_tensor, ingredients_tensor):
    category = str(category_tensor.numpy(), "utf-8")
    ingredients = str(ingredients_tensor.numpy(), "utf-8")
    one_hot_category_tensor = (
        data_pipeline.one_hot_encoder.get_category_one_hot_encoding(category)
    )
    one_hot_ingredients_tensor = (
        data_pipeline.one_hot_encoder.get_ingredients_one_hot_encoding(
            ingredients.split(",")
        )
    )
    return (
        tf.constant(one_hot_category_tensor, dtype=tf.uint8),
        tf.constant(one_hot_ingredients_tensor, dtype=tf.uint8),
    )

In [ ]:
def parse_function(x):
    encoded = tf.py_function(
        encode_category_ingredients,
        [x["category"], x["ingredients"]],
        [tf.uint8, tf.uint8],
    )
    return x["image_raw"], {
        "category_output": encoded[0],
        "calorie_output": x["calorie"],
        "carbs_output": x["carbs"],
        "protein_output": x["protein"],
        "fat_output": x["fat"],
        "ingredients_output": encoded[1],
    }

## Creating dataset for training ingredients and nutrients only

In [ ]:
# FULL DATASET
no_category_train, no_category_val = nutrition5k_builder.as_dataset(
    split=["train[:80%]", "train[80%:]"]
)

In [ ]:
print(f"Total training size : {no_category_train.cardinality().numpy()}")
print(f"Total validation size : {no_category_val.cardinality().numpy()}")

Total training size : 217463
Total validation size : 54366


In [ ]:
BATCH_SIZE = 256
no_category_train = (
    no_category_train.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
no_category_val = (
    no_category_val.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

## Creating dataset for training category, ingredients and nutrients

In [ ]:
# FULL DATASET
(
    recipes5k_food101_train,
    recipes5k_food101_validation,
) = recipes5k_food101_builder.as_dataset(split=["train[:80%]", "train[80%:]"])

# Introduce the generic category, to avoid a bias dataset, only 1k images (average image per category) is taken from nutrition5k dataset
(nutrition5k_category_train, nutrition5k_category_val) = nutrition5k_builder.as_dataset(
    split=["train[:800]", "train[800:1000]"]
)

train_dataset = recipes5k_food101_train.concatenate(nutrition5k_category_train)
validation_dataset = recipes5k_food101_validation.concatenate(nutrition5k_category_val)

In [ ]:
print(f"Total training size : {train_dataset.cardinality().numpy()}")
print(f"Total validation size : {validation_dataset.cardinality().numpy()}")

Total training size : 85461
Total validation size : 21365


In [ ]:
BATCH_SIZE = 256
train_dataset = (
    train_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    validation_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

# Model Development

In [ ]:
class BaseModel:
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        self.input_shape = input_shape
        self.total_food_category = total_food_category
        self.total_ingredients_category = total_ingredients_category
        self.model = None
        self.name = name
        self.model_config_name = model_config_name

    def build(
        self,
        independent_category_units,
        independent_ingredients_units,
        independent_protein_units,
        independent_calorie_units,
        independent_carbs_units,
        independent_fat_units,
        shared_units,
    ):
        raise NotImplementedError

    def compile_model(self, optimizer):
        assert self.model is not None
        category_classification_loss = keras.losses.CategoricalCrossentropy()
        calorie_regression_loss = keras.losses.MeanAbsoluteError()
        carbs_regression_loss = keras.losses.MeanAbsoluteError()
        protein_regression_loss = keras.losses.MeanAbsoluteError()
        fat_regression_loss = keras.losses.MeanAbsoluteError()
        ingredient_multilabel_loss = keras.losses.BinaryCrossentropy()
        category_classification_metrics = (
            [
                keras.metrics.CategoricalAccuracy(name="acc"),
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
            ],
        )

        calorie_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        carbs_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        protein_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        fat_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        ingredient_multilabel_metrics = (
            [
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
            ],
        )
        category_classification_loss_weights = 1.0
        ingredient_multilabel_loss_weights = 1.0
        calorie_regression_loss_weights = 1.0
        carbs_regression_loss_weights = 1.0
        protein_regression_loss_weights = 1.0
        fat_regression_loss_weights = 1.0

        self.model.compile(
            optimizer=optimizer,
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )

    def freeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = False

    def freeze_convolution_base(self):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetB1")
        submodel.trainable = False

    def get_callbacks(self):
        tensorboard_dir = f"/content/food_v5/main/models/logs/{self.name}/{self.model_config_name}"
        checkpoint_dir = f"/content/food_v5/main/temp/checkpoint/{self.name}/{self.model_config_name}"
        checkpoint_model_path = checkpoint_dir + "/model"
        checkpoint_weights_path = checkpoint_dir + "/weights/ckpt"
        checkpoint_model_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_model_path, monitor="val_loss", save_best_only=True, mode="min"
        )
        checkpoint_weights_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_weights_path,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True,
            mode="min",
        )
        reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1
        )
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_dir)
        early_stopping_callback = keras.callbacks.EarlyStopping("val_loss", patience=8)
        return [
            tensorboard_callback,
            early_stopping_callback,
            checkpoint_model_callback,
            checkpoint_weights_callback,
            reduce_lr_callback,
        ]

    # Layers from bottom to top (classifier)  #
    def get_input_layer(self):
        return keras.Input(shape=self.input_shape)

    def get_augmentation_layers(self):
        input_layer = keras.layers.Input(shape=self.input_shape)
        augmentation_layer = keras.layers.RandomFlip()(input_layer)
        augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
        return keras.Model(
            inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers"
        )

    def get_preprocess_layers(self, input_tensor):
        raise NotImplementedError

    def get_convolution_block(self):
        raise NotImplementedError

    def get_shared_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            num_units[0], activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

    def get_category_classification_layers(
        self, input_tensor, total_categories, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(
            num_units[0], activation="relu", name="category_dense_1"
        )(input_layer)
        x = keras.layers.BatchNormalization()(x)
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output_layer"
        )(x)
        output_model = keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )
        return output_model

    def get_calorie_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="calorie_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="calorie_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

    def get_carbs_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="carbs_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="carbs_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

    def get_protein_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="protein_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="protein_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        protein_regression_layers = keras.layers.Dense(1, name="protein_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    def get_fat_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="fat_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="fat_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        fat_regression_layers = keras.layers.Dense(1, name="fat_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

    def get_ingredients_multilabel_layers(
        self, input_tensor, total_ingredients, *num_units
    ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[0], activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[1], activation="relu", name="ingredients_dense_2"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[2], activation="relu", name="ingredients_dense_3"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output_layer"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    # End of layers  #

    def load_model(self, path):
        self.model = keras.models.load_model(path)

    def print_summary(self):
        assert (
            self.model is not None
        ), "Please run build_and_compile before printing summary."
        self.model.summary(expand_nested=True, show_trainable=True)

    def save_model(self, path):
        assert self.model is not None
        self.model.save(path, save_format="h5")

    def train_model(self, **kwargs):
        assert self.model is not None, "No model found."
        return self.model.fit(**kwargs, callbacks=self.get_callbacks())

    def unfreeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = False

    def unfreeze_convolution_base(self, fine_tune_at=0):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetb1")
        submodel.trainable = True
        for layer in submodel.layers[:fine_tune_at]:
            layer.trainable = False

## Flat Model

In [ ]:
class FlatModel(BaseModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        name,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            name,
            model_config_name,
        )

    def build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
    ):
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        prev_layer = self.get_convolution_block()(prev_layer)
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        self.model = model

### EfficientNetB1 (Best)

In [ ]:
class FlatEfficientNetB1Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

#### Train on ingredients dataset (nutrition5k)

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalIngredients",
)

flat_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_efficientnet.freeze_category_classification_layers()
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

27018416/27018416 [==============================] - 0s 0us/step
Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_1[0][0]']             Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_2 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y         |
|                         

In [ ]:
flat_efficientnet.train_model(
    x=no_category_train,
    validation_data=no_category_val,
    epochs=5,
    verbose=2,
)

Epoch 1/5


#### Train on main dataset (recipes5k + food101 + nutrition5k) with weights transfer from ingredients dataset

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategory",
)

flat_efficientnet.load_model("/content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalIngredients/model")
flat_efficientnet.unfreeze_category_classification_layers()
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_1[0][0]']             Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_2 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y         |
|                                                                                          

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset,
    validation_data=validation_dataset,
    epochs=5,
    verbose=2,
)

Epoch 1/5


KeyboardInterrupt: 

#### Train on main dataset (recipes5k + food101 + nutrition5k) without weights transfer

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransfer",
)

flat_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_efficientnet.compile_model(keras.optimizers.Adam())

flat_efficientnet.print_summary()

Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_12 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_12[0][0]']            Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_13 (InputLayer)      [(None, 224, 224, 3)]        0         []                            Y         |
|                                                                                          

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=20, verbose=2
)

#### Fine tune the convolution base without weights transfer

In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransferFineTuned",
)
flat_efficientnet.load_model(
    "/content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransfer/model"
)
flat_efficientnet.unfreeze_convolution_base(117)
flat_efficientnet.compile_model(keras.optimizers.Adam(learning_rate=1e-5))
flat_efficientnet.print_summary()

Model: "flat_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_12 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_12[0][0]']            Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_13 (InputLayer)      [(None, 224, 224, 3)]        0         []                            Y         |
|                                                                                          

In [ ]:
flat_efficientnet.train_model(
    x=train_dataset,
    validation_data=validation_dataset,
    epochs=10,
    verbose=2,
)

Epoch 1/10
334/334 - 956s - loss: 3.7696 - category_output_loss: 3.0041 - ingredients_output_loss: 0.0267 - calorie_output_loss: 0.5799 - carbs_output_loss: 0.0769 - protein_output_loss: 0.0232 - fat_output_loss: 0.0588 - category_output_acc: 0.3558 - category_output_precision: 0.5328 - category_output_recall: 0.2743 - ingredients_output_precision: 0.6276 - ingredients_output_recall: 0.3791 - calorie_output_MAE: 0.5799 - carbs_output_MAE: 0.0769 - protein_output_MAE: 0.0232 - fat_output_MAE: 0.0588 - val_loss: 2.9596 - val_category_output_loss: 2.2751 - val_ingredients_output_loss: 0.0224 - val_calorie_output_loss: 0.5227 - val_carbs_output_loss: 0.0660 - val_protein_output_loss: 0.0206 - val_fat_output_loss: 0.0529 - val_category_output_acc: 0.5023 - val_category_output_precision: 0.6452 - val_category_output_recall: 0.4410 - val_ingredients_output_precision: 0.7234 - val_ingredients_output_recall: 0.4859 - val_calorie_output_MAE: 0.5227 - val_carbs_output_MAE: 0.0660 - val_protein_ou

### MobileNetv3

In [ ]:
class FlatMobileNetv3Model(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_mobilenetv3",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.mobilenet_v3.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        mobilenet_v3_convolution_layers = keras.applications.MobileNetV3Large(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        mobilenet_v3_convolution_layers.trainable = False
        return mobilenet_v3_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
flat_mobilenetv3 = FlatMobileNetv3Model(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

flat_mobilenetv3.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_mobilenetv3.compile_model(keras.optimizers.Adam())
flat_mobilenetv3.print_summary()

12683000/12683000 [==============================] - 0s 0us/step
Model: "flat_mobilenetv3"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_23 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_23[0][0]']            Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_24 (InputLayer)      [(None, 224, 224, 3)]        0         []                            Y         |
|                            

In [ ]:
flat_mobilenetv3.train_model(
    x=no_category_train, validation_data=no_category_val, epochs=2, verbose=1
)

Epoch 1/2
850/850 [==============================] - 1435s 2s/step - loss: 1.0402 - category_output_loss: 0.2438 - ingredients_output_loss: 0.0648 - calorie_output_loss: 0.4851 - carbs_output_loss: 0.0824 - protein_output_loss: 0.0762 - fat_output_loss: 0.0879 - category_output_acc: 0.9702 - category_output_precision: 1.0000 - category_output_recall: 0.9282 - ingredients_output_precision: 0.0862 - ingredients_output_recall: 0.3566 - calorie_output_MAE: 0.4851 - carbs_output_MAE: 0.0824 - protein_output_MAE: 0.0762 - fat_output_MAE: 0.0879 - val_loss: 0.4663 - val_category_output_loss: 8.9781e-04 - val_ingredients_output_loss: 0.0072 - val_calorie_output_loss: 0.3377 - val_carbs_output_loss: 0.0433 - val_protein_output_loss: 0.0331 - val_fat_output_loss: 0.0442 - val_category_output_acc: 1.0000 - val_category_output_precision: 1.0000 - val_category_output_recall: 1.0000 - val_ingredients_output_precision: 0.8304 - val_ingredients_output_recall: 0.5059 - val_calorie_output_MAE: 0.3377 - 

### NASNet Mobile

In [ ]:
class FlatNASNetModel(FlatModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "flat_nasnet",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.nasnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        nasnet_convolution_layers = keras.applications.nasnet.NASNetMobile(
            input_shape=self.input_shape,
            include_top=False,
            weights="imagenet",
            pooling="avg",
        )
        nasnet_convolution_layers.trainable = False
        return nasnet_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
flat_nasnet = FlatNASNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="1-2048-(1-512)-[(2-64-32)-(2-64-32)-(2-64-32)-(2-64-32)]-(3-1024-512-256)-B256",
)

flat_nasnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)
flat_nasnet.compile_model(keras.optimizers.Adam())

flat_nasnet.print_summary()

19993432/19993432 [==============================] - 0s 0us/step
Model: "flat_nasnet"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_34 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_34[0][0]']            Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_35 (InputLayer)      [(None, 224, 224, 3)]        0         []                            Y         |
|                                 

In [ ]:
flat_nasnet.train_model(
    x=train_dataset, validation_data=validation_dataset, epochs=10, verbose=1
)

Epoch 1/10
334/334 [==============================] - 728s 2s/step - loss: 4.2636 - category_output_loss: 2.6840 - ingredients_output_loss: 0.1599 - calorie_output_loss: 1.0190 - carbs_output_loss: 0.1524 - protein_output_loss: 0.1276 - fat_output_loss: 0.1208 - category_output_acc: 0.3563 - category_output_precision: 0.7025 - category_output_recall: 0.1911 - ingredients_output_precision: 0.0561 - ingredients_output_recall: 0.2553 - calorie_output_MAE: 1.0190 - carbs_output_MAE: 0.1524 - protein_output_MAE: 0.1276 - fat_output_MAE: 0.1208 - val_loss: 3.0905 - val_category_output_loss: 2.2439 - val_ingredients_output_loss: 0.0296 - val_calorie_output_loss: 0.5780 - val_carbs_output_loss: 0.1032 - val_protein_output_loss: 0.0491 - val_fat_output_loss: 0.0867 - val_category_output_acc: 0.4349 - val_category_output_precision: 0.7374 - val_category_output_recall: 0.2704 - val_ingredients_output_precision: 0.7724 - val_ingredients_output_recall: 0.2031 - val_calorie_output_MAE: 0.5780 - val_

## Wide Slice Model

In [ ]:
class WideSliceModel(BaseModel):
    def build(
        self,
        shared_units,
        independent_category_units,
        independent_protein_units,
        independent_fat_units,
        independent_carbs_units,
        independent_calorie_units,
        independent_ingredients_units,
    ):
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        conv_layer = self.get_convolution_block()(prev_layer)
        wide_slice_layer = self.get_wideslice_conv_layers(prev_layer)(prev_layer)
        prev_layer = keras.layers.Concatenate()([conv_layer, wide_slice_layer])
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        self.model = model

    def get_wideslice_conv_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        rescale_layer = keras.layers.Rescaling(scale=1.0 / 255)(input_layer)
        wideslice_conv = keras.layers.Conv2D(filters=64, kernel_size=(5, 224))(
            rescale_layer
        )
        wideslice_conv = keras.layers.BatchNormalization()(wideslice_conv)
        wideslice_conv = keras.layers.ReLU()(wideslice_conv)
        wideslice_conv = keras.layers.GlobalMaxPool2D()(wideslice_conv)
        return keras.Model(
            inputs=input_layer, outputs=wideslice_conv, name="wideslice_conv"
        )

### EfficientNetB1

In [ ]:
class WideSliceEfficientNetModel(WideSliceModel):
    def __init__(
        self,
        input_shape,
        total_food_category,
        total_ingredients_category,
        model_config_name,
    ):
        super().__init__(
            input_shape,
            total_food_category,
            total_ingredients_category,
            "wideslice_efficientnetB1",
            model_config_name,
        )

    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers

#### Train on main dataset (recipes5k + food101 + nutrition5k)

In [ ]:
wideslice_efficientnet = WideSliceEfficientNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransfer",
)

wideslice_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
    independent_ingredients_units=[1024, 512, 256],
)
wideslice_efficientnet.compile_model(keras.optimizers.Adam())
wideslice_efficientnet.print_summary()

Model: "wideslice_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_45 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_45[0][0]']            Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_46 (InputLayer)      [(None, 224, 224, 3)]        0         []                            Y         |
|                                                                                     

In [ ]:
wideslice_efficientnet.train_model(
    x=train_dataset, epochs=25, verbose=2, validation_data=validation_dataset
)

Epoch 1/25
334/334 - 735s - loss: 2.7323 - category_output_loss: 1.9074 - ingredients_output_loss: 0.0311 - calorie_output_loss: 0.5849 - carbs_output_loss: 0.0832 - protein_output_loss: 0.0490 - fat_output_loss: 0.0767 - category_output_acc: 0.5218 - category_output_precision: 0.7672 - category_output_recall: 0.3737 - ingredients_output_precision: 0.7224 - ingredients_output_recall: 0.2617 - calorie_output_MAE: 0.5849 - carbs_output_MAE: 0.0832 - protein_output_MAE: 0.0490 - fat_output_MAE: 0.0767 - val_loss: 2.3216 - val_category_output_loss: 1.5960 - val_ingredients_output_loss: 0.0248 - val_calorie_output_loss: 0.5300 - val_carbs_output_loss: 0.0716 - val_protein_output_loss: 0.0334 - val_fat_output_loss: 0.0659 - val_category_output_acc: 0.5894 - val_category_output_precision: 0.7824 - val_category_output_recall: 0.4759 - val_ingredients_output_precision: 0.8025 - val_ingredients_output_recall: 0.3388 - val_calorie_output_MAE: 0.5300 - val_carbs_output_MAE: 0.0716 - val_protein_ou

KeyboardInterrupt: 

#### Transfer weights from fine tuned flat model

In [ ]:
fine_tuned_flat_efficientnet = keras.models.load_model(
    "/content/food_v5/main/temp/checkpoint/flat_efficientnetB1/finalCategoryNoTransferFineTuned/model"
)
fine_tuned_efficientnet_weights = fine_tuned_flat_efficientnet.get_layer(
    "efficientnetb1"
).get_weights()

In [ ]:
wideslice_efficientnet = WideSliceEfficientNetModel(
    input_shape=(224, 224, 3),
    total_food_category=len(data_pipeline.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(data_pipeline.one_hot_encoder.all_ingredients),
    model_config_name="finalCategoryNoTransferFineTuned",
)

wideslice_efficientnet.load_model(
    "/content/food_v5/main/temp/checkpoint/wideslice_efficientnetB1/finalCategoryNoTransfer/model"
)
wideslice_efficientnet.unfreeze_convolution_base(117)
wideslice_efficientnet.model.get_layer("efficientnetb1").set_weights(
    fine_tuned_efficientnet_weights
)
wideslice_efficientnet.compile_model(keras.optimizers.Adam(learning_rate=1e-5))
wideslice_efficientnet.print_summary()

Model: "wideslice_efficientnetB1"
_____________________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  Trainable  
 input_45 (InputLayer)       [(None, 224, 224, 3)]        0         []                            Y          
                                                                                                             
 augmentation_layers (Funct  (None, 224, 224, 3)          0         ['input_45[0][0]']            Y          
 ional)                                                                                                      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_46 (InputLayer)      [(None, 224, 224, 3)]        0         []                            Y         |
|                                                                                     

In [ ]:
wideslice_efficientnet.train_model(
    x=train_dataset,
    epochs=20,
    verbose=2,
    validation_data=validation_dataset,
)

Epoch 1/20
334/334 - 963s - loss: 1.6459 - category_output_loss: 1.0384 - ingredients_output_loss: 0.0173 - calorie_output_loss: 0.4629 - carbs_output_loss: 0.0583 - protein_output_loss: 0.0197 - fat_output_loss: 0.0494 - category_output_acc: 0.7089 - category_output_precision: 0.8267 - category_output_recall: 0.6308 - ingredients_output_precision: 0.8331 - ingredients_output_recall: 0.5608 - calorie_output_MAE: 0.4629 - carbs_output_MAE: 0.0583 - protein_output_MAE: 0.0197 - fat_output_MAE: 0.0494 - val_loss: 1.9342 - val_category_output_loss: 1.3315 - val_ingredients_output_loss: 0.0173 - val_calorie_output_loss: 0.4628 - val_carbs_output_loss: 0.0557 - val_protein_output_loss: 0.0189 - val_fat_output_loss: 0.0481 - val_category_output_acc: 0.6727 - val_category_output_precision: 0.7869 - val_category_output_recall: 0.6138 - val_ingredients_output_precision: 0.8389 - val_ingredients_output_recall: 0.5730 - val_calorie_output_MAE: 0.4628 - val_carbs_output_MAE: 0.0557 - val_protein_ou

In [ ]:
import shutil
import os
source_directory = '/content/food_v5/main'
destination_directory = '/content/drive/MyDrive/output2'

if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

shutil.copytree(source_directory, destination_directory, dirs_exist_ok=True)

print(f"Files from {source_directory} have been copied to {destination_directory}")

Files from /content/food_v5/main have been copied to /content/drive/MyDrive/output2
